In [1]:
!pip install transformers[torch] -q
!pip install numpy matplotlib -q

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import pipeline
import re

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
model1=AutoModelForMaskedLM.from_pretrained('sampathlonka/San-BERT') 
model1

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [6]:
tokenizer1=AutoTokenizer.from_pretrained('sampathlonka/San-BERT')
tokenizer1

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/951k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='sampathlonka/San-BERT', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [26]:
from glob import glob
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

In [27]:
custom_tokenizer = Tokenizer(models.BPE())
custom_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
custom_tokenizer.decoder = decoders.ByteLevel()
custom_tokenizer.post_processor = processors.ByteLevel()

In [28]:
tokenized_data = []

# Specify the pattern to match your file names
file_pattern = "/kaggle/input/finetune/*.txt"  # Update this to your actual path

# Get a list of file paths that match the pattern
file_paths = glob(file_pattern)

In [31]:
file_paths

['/kaggle/input/finetune/3.txt',
 '/kaggle/input/finetune/10.txt',
 '/kaggle/input/finetune/19.txt',
 '/kaggle/input/finetune/5.txt',
 '/kaggle/input/finetune/7.txt',
 '/kaggle/input/finetune/8.txt',
 '/kaggle/input/finetune/18.txt',
 '/kaggle/input/finetune/17.txt',
 '/kaggle/input/finetune/11.txt',
 '/kaggle/input/finetune/0.txt',
 '/kaggle/input/finetune/9.txt',
 '/kaggle/input/finetune/16.txt',
 '/kaggle/input/finetune/1.txt',
 '/kaggle/input/finetune/13.txt',
 '/kaggle/input/finetune/14.txt',
 '/kaggle/input/finetune/12.txt',
 '/kaggle/input/finetune/15.txt',
 '/kaggle/input/finetune/2.txt',
 '/kaggle/input/finetune/4.txt',
 '/kaggle/input/finetune/6.txt']

In [30]:
trainer = trainers.BpeTrainer(special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"])

# Iterate through each file and add it to the trainer
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        # Add the text to the trainer
        custom_tokenizer.add_tokens([text])

In [32]:
text

'अच्युताष्टकम्\nअथ श्री अच्युताष्टकम्\nअच्युतं केशवं रामनारायणं\nकृष्णदामोदरं वासुदेवं हरिं\nश्रीधरं माधवं गोपिकावल्लभं\nजानकीनायकं रामचन्द्रं भजे ।\nअच्युतं केशवं सत्यभामाधवं\nमाधवं श्रीधरं राधिकाराधितम्\nइन्दिरा मन्दिरं चेतसा सुन्दरं\nदेवकीनन्दनं नन्दनं सन्दधे ।\nविष्णवे जिष्णवे शङ्खिने चक्रिणे\nरुक्मिनी रागिने जानकी जानये\nवल्लवी वल्लभायाऽर्चितायात्मने\nकंस विध्वंसिने वंशिने ते नमः ।\nकृष्ण गोविन्द हे राम नारायण \nश्रीपते वासुदेवाजित श्रीनिधे\nअच्युतानन्त हे माधवाधोक्षज\nद्वारका नायक द्रौ पदी रक्षक ।\nराक्षसक्षोभितः सीतयाशोभितो \nदण्डकारण्य भू पुण्यता कारणः\nलक्ष्मणेनान्वितो वानरैः सेवितो\nअगस्त्यसम्पूजितो राघवः पातु माम् \nधेनुकारिष्टकोऽनिष्टकृद्द्वेषिणां\nकेशिहा कंसहृद्वंशिकावादकः\nपूतनाकोपकः सूरजा खेलनो\nबाल गोपालकः पातु मां सर्वदा ।\nविद्युदुद्ध्योतवान् प्रस्फुरद्वाससं\nप्रावृडम्भोदवत्प्रोल्लसद्विग्रहम्\nवन्यया मालया शोभितोरस्थलं\nलोहिताङ्घ्रिद्वयं वारिजाक्षं भजे ।\nकुञ्चितैः कुन्तलैर्भ्राजमानाननं\nरत्नमौलिं लसत्कुण्डलं गण्डयोः\nहारकेयूरकं कङ्कण प्रोज्ज्वलं \nकिङ्किणी मञ्जुलं श्

In [34]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

# Initialize a tokenizer
custom_tokenizer = Tokenizer(models.BPE())
custom_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
custom_tokenizer.decoder = decoders.ByteLevel()
custom_tokenizer.post_processor = processors.ByteLevel()

In [36]:
initial_vocab_size = len(custom_tokenizer.get_vocab())

In [43]:
tokenized_data2 = []

# Specify the pattern to match your file names
file_pattern2 = "/kaggle/input/isittrained/*.txt"  # Update this to your actual path

# Get a list of file paths that match the pattern
file_paths2 = glob(file_pattern2)

In [44]:
file_paths2

['/kaggle/input/isittrained/21.txt',
 '/kaggle/input/isittrained/23.txt',
 '/kaggle/input/isittrained/20.txt',
 '/kaggle/input/isittrained/26.txt',
 '/kaggle/input/isittrained/22.txt',
 '/kaggle/input/isittrained/28.txt',
 '/kaggle/input/isittrained/25.txt',
 '/kaggle/input/isittrained/29.txt',
 '/kaggle/input/isittrained/27.txt',
 '/kaggle/input/isittrained/24.txt']

In [45]:
trainer2 = trainers.BpeTrainer(special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"])

# Iterate through each file and add it to the trainer
for file_path in file_paths2:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        # Add the text to the trainer
        custom_tokenizer.add_tokens([text])

In [46]:
final_vocab_size = len(custom_tokenizer.get_vocab())

if final_vocab_size > initial_vocab_size:
    print("Tokenizer is trained.")
else:
    print("Tokenizer is not trained.")

Tokenizer is trained.
